In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [6]:
device = "cuda:2"

In [7]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/factRuEval-2016/dev.csv",
    valid_df_path="/home/eartemov/ae/work/factRuEval-2016/test.csv",
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels5.txt",
    clear_cache=True,
    device=device
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [4]:
from modules.models.bert_models import BERTBiLSTMNCRF

In [8]:
model = BERTBiLSTMNCRF.create(
    len(data.train_ds.idx2label), lstm_dropout=0., crf_dropout=0.3, nbest=len(data.train_ds.idx2label)-1, device=device)

build CRF...


In [9]:
from modules.train.train import NerLearner

In [10]:
num_epochs = 100

In [11]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMNCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [12]:
model.get_n_trainable_params()

2235259

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

In [22]:
from modules.data.bert_data import get_data_loader_for_predict

In [23]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [24]:
preds = learner.predict(dl)

In [25]:
from sklearn_crfsuite.metrics import flat_classification_report

In [26]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [27]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [28]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=learner.sup_labels, digits=4)

In [29]:
print(tokens_report)

              precision    recall  f1-score   support

         I_O     0.9777    0.9886    0.9831     48875
       I_LOC     0.8996    0.7996    0.8467      1557
       I_PER     0.9373    0.9626    0.9498      2112
       I_ORG     0.8709    0.7281    0.7931      3865

   micro avg     0.9681    0.9646    0.9663     56409
   macro avg     0.9214    0.8697    0.8932     56409
weighted avg     0.9667    0.9646    0.9651     56409



In [30]:
from modules.analyze_utils.main_metrics import precision_recall_f1

In [32]:
results = precision_recall_f1(true_labels, pred_labels)

processed 56409 tokens with 7534 phrases; found: 6784 phrases; correct: 6092.

precision:  89.80%; recall:  80.86%; FB1:  85.10

	LOC: precision:  89.96%; recall:  79.96%; F1:  84.67 1384

	ORG: precision:  87.09%; recall:  72.81%; F1:  79.31 3231

	PER: precision:  93.73%; recall:  96.26%; F1:  94.98 2169


